# Import Packages and Files

In [1]:
import torch.nn as nn
import dataloader as dl
import filemanager as fm
import NonLinearClassifier as NLC
import RotNet as RN
import trainer as tr

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 8000, 128)

# Train RotNet for Rotation Task

In [4]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [7]:
# train network with own implemented metric
rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [1, 2, 0, 0], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270']) # [60, 120, 160, 200]

[1, 60] loss: 0.584
[1, 120] loss: 0.577
[1, 180] loss: 0.573
[1, 240] loss: 0.559
[1, 300] loss: 0.550
Epoch: 1 -> Loss: 0.640613377094
Epoch: 1 -> Validation Accuracy: 77.703125
[2, 60] loss: 0.431
[2, 120] loss: 0.415
[2, 180] loss: 0.406
[2, 240] loss: 0.386
[2, 300] loss: 0.409
Epoch: 2 -> Loss: 0.282387077808
Epoch: 2 -> Validation Accuracy: 84.1375


In [5]:
# train network with metric from paper
rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [2, 3, 0, 0], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270'], use_paper_metric=True)

[1, 60] loss: 1.094
[1, 120] loss: 0.947
[1, 180] loss: 0.856
[1, 240] loss: 0.839
[1, 300] loss: 0.794
Epoch: 1 -> Loss: 0.92122232914
Epoch: 1 -> Validation Accuracy: 68.4585813492
[2, 60] loss: 0.777
[2, 120] loss: 0.713
[2, 180] loss: 0.677
[2, 240] loss: 0.652
[2, 300] loss: 0.664
Epoch: 2 -> Loss: 0.476222127676
Epoch: 2 -> Validation Accuracy: 75.1705109127
[3, 60] loss: 0.536
[3, 120] loss: 0.516
[3, 180] loss: 0.511
[3, 240] loss: 0.476
[3, 300] loss: 0.503
Epoch: 3 -> Loss: 0.537952363491
Epoch: 3 -> Validation Accuracy: 80.9151785714


# Save results for Rotation Task

In [10]:
rot_result = [rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch]
paper_rot_result = [rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch]

fm.save_variable(rot_result, 'rotation')
fm.save_variable(paper_rot_result, 'rotation_paper')

# Load results from previous trained Rotation Task

In [11]:
load = False

if load:
    rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = fm.load_variable('rotation')
    rot_paper_loss_log, rot_paper_accuracy_log, rot_paper_max_accuracy, rot_paper_best_epoch = fm.load_variable(
                                                                                                        'rotation_paper')

# Train classifier on Conv Block 1

In [4]:
# initialize classifier
clf_1 = NLC.NonLinearClassifier(10, 96*16*16)